In [1]:
import asyncio
import itertools
import logging
import random
import nest_asyncio
import pandas as pd

nest_asyncio.apply()  #

In [2]:
import torch

torch.cuda.is_available()

True

In [3]:
import os
from pathlib import Path
import tempfile

from shutil import rmtree
from typing import List, Optional, Union

from nuplan.planning.script.run_simulation import main as main_simulation

In [4]:
%env NUPLAN_DATA_ROOT=../../../datasets/nuplan/dataset
%env NUPLAN_MAPS_ROOT=../../data/nuplan/maps
%env NUPLAN_EXP_ROOT=../../data/nuplan/exp
%env NUPLAN_DEVKIT_ROOT=../../nuplan-devkit/
%pwd

env: NUPLAN_DATA_ROOT=../../../datasets/nuplan/dataset
env: NUPLAN_MAPS_ROOT=../../data/nuplan/maps
env: NUPLAN_EXP_ROOT=../../data/nuplan/exp
env: NUPLAN_DEVKIT_ROOT=../../nuplan-devkit/


'/home/ehdykhne/nuplan-devkit/experiments'

In [5]:
import hydra
import pytorch_lightning as pl
from omegaconf import DictConfig, OmegaConf

from nuplan.common.utils.s3_utils import is_s3_path
from nuplan.planning.script.builders.simulation_builder import build_simulations
from nuplan.planning.script.builders.simulation_callback_builder import (
    build_callbacks_worker,
    build_simulation_callbacks,
)
from nuplan.planning.script.utils import (
    run_runners,
    set_default_path,
    set_up_common_builder,
)
from nuplan.planning.simulation.planner.abstract_planner import AbstractPlanner

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# If set, use the env. variable to overwrite the default dataset and experiment paths
set_default_path()

# If set, use the env. variable to overwrite the Hydra config
CONFIG_PATH = os.getenv("NUPLAN_HYDRA_CONFIG_PATH", "config/simulation")

if os.environ.get("NUPLAN_HYDRA_CONFIG_PATH") is not None:
    CONFIG_PATH = os.path.join("../../../../", CONFIG_PATH)

if os.path.basename(CONFIG_PATH) != "simulation":
    CONFIG_PATH = os.path.join(CONFIG_PATH, "simulation")
CONFIG_NAME = "default_simulation"

In [6]:
# list of selected scenario tokens
from tokens import selected_scenario_tokens, modifiable_scenario_tokens

print(selected_scenario_tokens[0:3])
print(modifiable_scenario_tokens[0:3])

selected_scenario_tokens = ["8b80aa05d88b518b"]

['d6234553edc95d50', '98ddd7851e445b7a', '5ac5795b9b5851e8']
['014c7f3502d4597d', '02828821ef045829', '02fa9b8d43465e0d']


In [7]:
from tutorials.utils.tutorial_utils import construct_simulation_hydra_paths

# Location of paths with all simulation configs
BASE_CONFIG_PATH = os.path.join(
    os.getenv("NUPLAN_TUTORIAL_PATH", ""), "../nuplan/planning/script"
)
simulation_hydra_paths = construct_simulation_hydra_paths(BASE_CONFIG_PATH)

In [8]:
left_turn_types = [
    "stopping_at_crosswalk",
    "high_lateral_acceleration",
    "near_multiple_vehicles",
    "on_stopline_stop_sign",
    "near_long_vehicle",
    "unknown",
    "near_barrier_on_driveable",
    "near_high_speed_vehicle",
]
scenario_types = [
    "starting_left_turn",
    # "accelerating_at_traffic_light_with_lead",
    # "crossed_by_bike",
    # "crossed_by_vehicle",
    # "on_intersection",
    # "on_stopline_crosswalk",
    # "on_stopline_stop_sign",
    # "on_stopline_traffic_light",
    # "on_traffic_light_intersection",
    # "starting_protected_cross_turn",
    # "starting_protected_noncross_turn",
    "starting_right_turn",
    "starting_straight_stop_sign_intersection_traversal",
    "starting_straight_traffic_light_intersection_traversal",
    # "starting_u_turn",
    "starting_unprotected_cross_turn",
    "starting_unprotected_noncross_turn",
    # "stationary_at_crosswalk",
    # "stationary_at_traffic_light_with_lead",
    # "stationary_at_traffic_light_without_lead",
    # "traversing_crosswalk",
    "traversing_intersection",
    "traversing_traffic_light_intersection",
]

left_turn_potential = [
    "starting_left_turn",
    "starting_unprotected_cross_turn",
]

# scenario_types = ['stationary_at_traffic_light_without_lead']

scenario_builder = "val"  # [nuplan (uses trainval), nuplan_mini, test, val, train_boston, train_pittsburgh, train_singapore]
DATASET_PARAMS = [
    f"scenario_builder={scenario_builder}",
    "scenario_filter=all_scenarios",  # [all_scenarios, val14_split]
    f"scenario_filter.scenario_types={scenario_types}",  # there are 70 scenario types in the trainingset and 58 in the validation set including "unknown" which make up the majority
    "scenario_filter.ego_displacement_minimum_m=10",  # use scenarios where the ego vehicle moves at least 10m
    #    'scenario_filter.remove_invalid_goals=true',  # remove scenarios where the goal is not invalid
    # "scenario_filter.ego_start_speed_threshold=5",  # Exclusive threshold that the ego's speed must rise above (meters per second) for scenario to be kept
    #    'scenario_filter.stop_speed_threshold=10',  # Inclusive threshold that the ego's speed must fall below (meters per second) for scenario to be kept:
    "scenario_filter.map_names=[sg-one-north, us-ma-boston, us-pa-pittsburgh-hazelwood, us-nv-las-vegas-strip]",  # select multiple [sg-one-north, us-ma-boston, us-pa-pittsburgh-hazelwood, us-nv-las-vegas-strip]
    # "scenario_filter.limit_total_scenarios=0.05",  # use n total scenarios if int, or if float smaller than 1, use n as a fraction of total scenarios (changes sampling frequency, unchanged leaves the frequency at 20Hz)
    "scenario_filter.num_scenarios_per_type=70",  # use 10 scenarios per scenario type
    # f"scenario_filter.scenario_tokens={selected_scenario_tokens}",  # List of scenarios to include (token)
    #'scenario_filter.log_names=["2021.08.24.20.03.01_veh-45_01091_01622"]',  # specific scenrios to simulate
    # turn off if you are selecting scenarios manually. turn on for initial filtering (0.05 gives us 1 scenario per second)
]

In [9]:
urban_ckpt = "/home/ehdykhne/nuplan-devkit/experiments/pretrained_checkpoints/urbandriver_checkpoint.ckpt"
gc_pgp_ckpt = "/home/ehdykhne/nuplan-devkit/experiments/pretrained_checkpoints/gc_pgp_checkpoint.ckpt"
hybrid_ckpt = "/home/ehdykhne/nuplan-devkit/experiments/pretrained_checkpoints/pdm_offset_checkpoint.ckpt"
#'/home/ehdykhne/nuplan-devkit/experiments/pretrained_checkpoints/urbandriver_checkpoint.ckpt'
#'/home/ehdykhne/Repos/nuplan-devkit/experiments/pretrained_checkpoints/urbandriver_checkpoint.ckpt'
# Initialize configuration management system
hydra.core.global_hydra.GlobalHydra.instance().clear()  # reinitialize hydra if already initialized
hydra.initialize(config_path=simulation_hydra_paths.config_path)

# Compose the configuration
print(simulation_hydra_paths.config_name)
cfg_occ = hydra.compose(
    config_name=simulation_hydra_paths.config_name,
    overrides=[
        "+simulation=closed_loop_multiagent",  # [open_loop_boxes, closed_loop_nonreactive_agents, closed_loop_reactive_agents, closed_loop_multiagent]
        "planner=pdm_closed_planner",  # [ml_planner, pdm_hybrid_planner, pdm_closed_planner, idm_planner, log_future_planner, simple_planner]
        # "model=gc_pgp_model",
        # "model.aggregator.pre_train=false",
        # "planner.ml_planner.model_config=${model}",
        # f"planner.ml_planner.checkpoint_path={gc_pgp_ckpt}",
        # f"planner.pdm_closed_planner.checkpoint_path={hybrid_ckpt}",
        # "+observation.model=gc_pgp_model",
        # "+observation.model.aggregator.pre_train=false",
        # "+observation.model_config=${model}",
        # f"+observation.checkpoint_path={gc_pgp_ckpt}",
        f"observation.planner_type=pdm_closed",
        # # f"observation.pdm_hybrid_ckpt={hybrid_ckpt}",
        f"observation.occlusion_cfg.occlusion=True",
        f"observation.occlusion_cfg.manager_type=wedge",
        f"+observation.occlusion_cfg.uncloak_reaction_time=1.5",
        f"+observation.occlusion_cfg.notice_threshold=1.0",
        "+modify_scenario_simulations=true",
        "+modifier_types=[conflict-and-occlusion-injection]",  # can select multiple: [occlusion-injection, left-and-right, oncoming-left-turn-and-occlusion-injection, conflict-and-occlusion-injection]
        "+occlusion=True",
        "+occlusion.manager_type=wedge",  # options: [range, shadow, wedge]
        "+occlusion.uncloak_reaction_time=1.5",  # 0.1
        "+occlusion.notice_threshold=1.0",  # 0.1
        "worker=ray_distributed",  # [sequential, ray_distributed]
        "hydra.searchpath=[pkg://tuplan_garage.planning.script.config.common, pkg://tuplan_garage.planning.script.config.simulation, pkg://nuplan.planning.script.config.common, pkg://nuplan.planning.script.experiments]",
        *DATASET_PARAMS,
    ],
)

output_folder_occ = cfg_occ.output_dir
print('output_folder_occ = "' + output_folder_occ + '"')


# closed mixed for simulation
# "+simulation=closed_loop_multiagent",  # [open_loop_boxes, closed_loop_nonreactive_agents, closed_loop_reactive_agents, closed_loop_multiagent]
# "planner=pdm_closed_planner",  # [ml_planner, pdm_hybrid_planner, pdm_closed_planner, idm_planner, log_future_planner, simple_planner]
# # "model=gc_pgp_model",
# # "model.aggregator.pre_train=false",
# # "planner.ml_planner.model_config=${model}",
# # f"planner.ml_planner.checkpoint_path={gc_pgp_ckpt}",
# # f"planner.pdm_closed_planner.checkpoint_path={hybrid_ckpt}",
# # "+observation.model=gc_pgp_model",
# # "+observation.model.aggregator.pre_train=false",
# # "+observation.model_config=${model}",
# # f"+observation.checkpoint_path={gc_pgp_ckpt}",
# f"observation.planner_type=pdm_closed",
# # # f"observation.pdm_hybrid_ckpt={hybrid_ckpt}",
# f"observation.occlusion_cfg.occlusion=true",
# f"observation.occlusion_cfg.manager_type=wedge",
# f"+observation.occlusion_cfg.uncloak_reaction_time=1.5",
# f"+observation.occlusion_cfg.notice_threshold=1.0",
# "+modify_scenario_simulations=true",
# "+modifier_types=[occlusion-injection]",  # can select multiple: [occlusion-injection, left-and-right]
# "+occlusion=true",
# "+occlusion.manager_type=wedge",  # options: [range, shadow, wedge]
# "+occlusion.uncloak_reaction_time=1.0",
# "+occlusion.notice_threshold=0.1",
# "worker=ray_distributed",  # [sequential, ray_distributed]
# "hydra.searchpath=[pkg://tuplan_garage.planning.script.config.common, pkg://tuplan_garage.planning.script.config.simulation, pkg://nuplan.planning.script.config.common, pkg://nuplan.planning.script.experiments]",
# *DATASET_PARAMS,

# openloop config for finding risky scenarios
# f"scenario_builder={scenario_builder}",
# "scenario_filter=all_scenarios",  # [all_scenarios, val14_split]
# # f"scenario_filter.scenario_types={left_turn_types}",  # there are 70 scenario types in the trainingset and 58 in the validation set including "unknown" which make up the majority
# "scenario_filter.ego_displacement_minimum_m=10",  # use scenarios where the ego vehicle moves at least 10m
# #    'scenario_filter.remove_invalid_goals=true',  # remove scenarios where the goal is not invalid
# "scenario_filter.ego_start_speed_threshold=5",  # Exclusive threshold that the ego's speed must rise above (meters per second) for scenario to be kept
# #    'scenario_filter.stop_speed_threshold=10',  # Inclusive threshold that the ego's speed must fall below (meters per second) for scenario to be kept:
# "scenario_filter.map_names=[sg-one-north, us-ma-boston, us-pa-pittsburgh-hazelwood]",  # select multiple [sg-one-north, us-ma-boston, us-pa-pittsburgh-hazelwood, us-nv-las-vegas-strip]
# # "scenario_filter.num_scenarios_per_type=300",  # use 10 scenarios per scenario type
# # f"scenario_filter.scenario_tokens={selected_scenario_tokens}",  # List of scenarios to include (token)
# #'scenario_filter.log_names=["2021.08.24.20.03.01_veh-45_01091_01622"]',  # specific scenrios to simulate
# "scenario_filter.limit_total_scenarios=0.01",  # use n total scenarios if int, or if float smaller than 1, use n as a fraction of total scenarios (changes sampling frequency, unchanged leaves the frequency at 20Hz)
# # turn off if you are selecting scenarios manually. turn on for initial filtering (0.05 gives us 1 scenario per second)


# "+simulation=open_loop_boxes",  # [open_loop_boxes, closed_loop_nonreactive_agents, closed_loop_reactive_agents, closed_loop_multiagent]
#         "planner=log_future_planner",  # [ml_planner, pdm_hybrid_planner, gc_pgp_planner, idm_planner, log_future_planner, simple_planner]
#         "ego_controller=perfect_tracking_controller",
#         "worker=ray_distributed",  # [sequential, ray_distributed]
#         "+occlusion=true",  # [true, false]
#         "+occlusion.manager_type=wedge",  # options: [range, shadow, wedge]
#         "+occlusion.uncloak_reaction_time=1.5",
#         "+occlusion.notice_threshold=1.0",
#         # "hydra.searchpath=[pkg://tuplan_garage.planning.script.config.common, pkg://tuplan_garage.planning.script.config.simulation, pkg://nuplan.planning.script.config.common, pkg://nuplan.planning.script.experiments]",
#         *DATASET_PARAMS,


# output folders for runs:
# closed_loop_reactive_agents, wedge occlusions, urban_planner
# output_folder = "../../data/nuplan/exp/exp/simulation/closed_loop_reactive_agents/2023.12.09.21.59.48"
# overall score: 0.5041, ego_at_fault_collisions: 363

# closed_loop_reactive_agents, occlusions = False, urban_planner
# output_folder = "../../data/nuplan/exp/exp/simulation/closed_loop_reactive_agents/2023.12.09.23.21.13"
# overall score: 0.5041, ego_at_fault_collisions: 357

# closed_loop_nonreactive_agents, occlusions = False, urban_planner
# output_folder = "../../data/nuplan/exp/exp/simulation/closed_loop_nonreactive_agents/2023.12.10.02.01.14"
# output_folder = "../../data/nuplan/exp/exp/simulation/closed_loop_nonreactive_agents/2023.12.10.07.32.41"
# overall score: , ego_at_fault_collisions:

# closed_loop_nonreactive_agents, wedge occlusions, urban_planner
# output_folder = "../../data/nuplan/exp/exp/simulation/closed_loop_nonreactive_agents/2023.12.10.08.29.23"
# overall score: , ego_at_fault_collisions:

# closed_loop_reactive_agents, wedge occlusions, gc_pgp
# output_folder = "../../data/nuplan/exp/exp/simulation/closed_loop_reactive_agents/2023.12.10.09.31.44"
# overall score: , ego_at_fault_collisions:

# 'closed_loop_reactive_agents, occlusions = False, gc_pgp'
# output_folder = "../../data/nuplan/exp/exp/simulation/closed_loop_reactive_agents/2023.12.11.00.41.30"

# 'closed_loop_reactive_agents, occlusions = False, pdm_hybrid'
# output_folder = "../../data/nuplan/exp/exp/simulation/closed_loop_reactive_agents/2023.12.13.12.21.28"

# 'closed_loop_reactive_agents,  wedge occlusions, pdm_hybrid'
# output_folder = "../../data/nuplan/exp/exp/simulation/closed_loop_reactive_agents/2023.12.13.13.07.48"

# # be8c7a230b9c5b4e_oncoming_vehicle_injection_for_left_turn_and_occlusion_injection_0
# # output_folder = "../../data/nuplan/exp/exp/simulation/closed_loop_multiagent/2024.02.14.14.14.30"
# output_folder = "../../data/nuplan/exp/exp/simulation/closed_loop_multiagent/2024.02.14.14.38.02"

default_simulation
output_folder_occ = "../../data/nuplan/exp/exp/simulation/closed_loop_multiagent/2024.03.06.00.31.10"


# Now we can run with the following

In [10]:
# Run the simulation loop (real-time visualization not yet supported, see next section for visualization)
main_simulation(cfg_occ)

Global seed set to 0
INFO:nuplan.planning.script.builders.main_callback_builder:Building MultiMainCallback...
INFO:nuplan.planning.script.builders.main_callback_builder:Building MultiMainCallback: 4...DONE!


2024-03-06 00:31:10,927 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/worker_pool_builder.py:19}  Building WorkerPool...
2024-03-06 00:31:10,929 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/utils/multithreading/worker_ray.py:78}  Starting ray local!


2024-03-06 00:31:12,555	INFO worker.py:1636 -- Started a local Ray instance.


2024-03-06 00:31:13,261 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/utils/multithreading/worker_pool.py:101}  Worker: RayDistributed
2024-03-06 00:31:13,262 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/utils/multithreading/worker_pool.py:102}  Number of nodes: 1
Number of CPUs per node: 32
Number of GPUs per node: 4
Number of threads across all nodes: 32
2024-03-06 00:31:13,262 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/worker_pool_builder.py:27}  Building WorkerPool...DONE!
2024-03-06 00:31:13,262 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/folder_builder.py:32}  Building experiment folders...
2024-03-06 00:31:13,262 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/folder_builder.py:35}  

	Folder where all results are stored: ../../data/nuplan/exp/exp/simulation/closed_loop_multiagent/2024.03.06.00.31.10

2024-03-06 00:31:13,264 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/folder_build

Ray objects: 100%|██████████| 32/32 [00:07<00:00,  4.11it/s]
(raylet) [2024-03-06 00:31:22,526 E 458942 458954] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-03-06_00-31-10_957136_458634 is over 95% full, available space: 631398117376; capacity: 16125711347712. Object creation will fail if spilling is required.


...

2024-03-06 00:36:50,508 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/simulation_builder.py:203}  Created 996 modified scenarios from 499 scenarios, 165 of which were modifiable.
2024-03-06 00:36:50,509 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/simulation_builder.py:204}  Building simulations...DONE!
2024-03-06 00:36:50,511 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/run_simulation.py:113}  Running simulation...
2024-03-06 00:36:50,511 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/utils.py:138}  Executing runners...
2024-03-06 00:36:50,511 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/simulation/runner/executor.py:82}  Starting 996 simulations using RayDistributed!


Ray objects:   0%|          | 0/996 [00:00<?, ?it/s](raylet) [2024-03-06 00:36:52,756 E 458942 458954] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-03-06_00-31-10_957136_458634 is over 95% full, available space: 631396720640; capacity: 16125711347712. Object creation will fail if spilling is required.
(raylet) [2024-03-06 00:37:02,766 E 458942 458954] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-03-06_00-31-10_957136_458634 is over 95% full, available space: 631396659200; capacity: 16125711347712. Object creation will fail if spilling is required.
(raylet) [2024-03-06 00:37:12,776 E 458942 458954] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-03-06_00-31-10_957136_458634 is over 95% full, available space: 631396630528; capacity: 16125711347712. Object creation will fail if spilling is required.
(raylet) [2024-03-06 00:37:22,785 E 458942 458954] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-03-06_00-31-10_957136_458634 is over 95% fu

2024-03-06 02:25:32,626 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/simulation/runner/executor.py:129}  Number of successful simulations: 996
2024-03-06 02:25:32,627 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/simulation/runner/executor.py:130}  Number of failed simulations: 0
2024-03-06 02:25:32,627 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/utils.py:147}  Finished executing runners!
2024-03-06 02:25:32,705 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/utils.py:74}  Saved runner reports to ../../data/nuplan/exp/exp/simulation/closed_loop_multiagent/2024.03.06.00.31.10/runner_report.parquet
2024-03-06 02:25:32,705 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/simulation/main_callback/time_callback.py:27}  Simulation duration: 01:54:21 [HH:MM:SS]


(raylet) [2024-03-06 02:25:38,042 E 458942 458954] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-03-06_00-31-10_957136_458634 is over 95% full, available space: 615829454848; capacity: 16125711347712. Object creation will fail if spilling is required.


2024-03-06 02:25:45,950 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/simulation/main_callback/metric_file_callback.py:79}  Metric files integration: 00:00:13 [HH:MM:SS]
2024-03-06 02:25:46,241 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/simulation/main_callback/metric_aggregator_callback.py:58}  Running metric aggregator: closed_loop_multiagent_weighted_average
2024-03-06 02:25:47,239 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/simulation/main_callback/metric_aggregator_callback.py:69}  Metric aggregator: 00:00:01 [HH:MM:SS]
2024-03-06 02:25:47,450 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/simulation/main_callback/metric_summary_callback.py:344}  Metric summary: 00:00:00 [HH:MM:SS]
2024-03-06 02:25:47,453 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/run_simulation.py:115}  Finished running simulation!


# Now we set up the second running regime

In [11]:
#########################################

path = output_folder_occ + "/metrics/"
file_path_1 = path + "no_ego_at_fault_collisions.parquet"
df = pd.read_parquet(file_path_1)
df = df.loc[df["no_ego_collisions_stat_value"] == False]
df = df["scenario_name"]


scenario_list = df.tolist()
print(f"+scenarios_to_check_in_alternate_regime={scenario_list}")

hydra.core.global_hydra.GlobalHydra.instance().clear()  # reinitialize hydra if already initialized
hydra.initialize(config_path=simulation_hydra_paths.config_path)
cfg_unocc = hydra.compose(
    config_name=simulation_hydra_paths.config_name,
    overrides=[
        "+simulation=closed_loop_multiagent",  # [open_loop_boxes, closed_loop_nonreactive_agents, closed_loop_reactive_agents, closed_loop_multiagent]
        "planner=pdm_closed_planner",  # [ml_planner, pdm_hybrid_planner, pdm_closed_planner, idm_planner, log_future_planner, simple_planner]
        # "model=gc_pgp_model",
        # "model.aggregator.pre_train=false",
        # "planner.ml_planner.model_config=${model}",
        # f"planner.ml_planner.checkpoint_path={gc_pgp_ckpt}",
        # f"planner.pdm_closed_planner.checkpoint_path={hybrid_ckpt}",
        # "+observation.model=gc_pgp_model",
        # "+observation.model.aggregator.pre_train=false",
        # "+observation.model_config=${model}",
        # f"+observation.checkpoint_path={gc_pgp_ckpt}",
        f"observation.planner_type=pdm_closed",
        # # f"observation.pdm_hybrid_ckpt={hybrid_ckpt}",
        f"observation.occlusion_cfg.occlusion=false",
        "+modify_scenario_simulations=true",
        "+second_testing_round=true",
        f"+scenarios_to_check_in_alternate_regime={scenario_list}",
        "+modifier_types=[conflict-and-occlusion-injection]",  # can select multiple: [occlusion-injection, left-and-right, oncoming-left-turn-and-occlusion-injection, conflict-and-occlusion-injection]
        "+occlusion=false",
        "worker=ray_distributed",  # [sequential, ray_distributed]
        "hydra.searchpath=[pkg://tuplan_garage.planning.script.config.common, pkg://tuplan_garage.planning.script.config.simulation, pkg://nuplan.planning.script.config.common, pkg://nuplan.planning.script.experiments]",
        *DATASET_PARAMS,
    ],
)

output_folder_unocc = cfg_unocc.output_dir
print('output_folder_unocc = "' + output_folder_unocc + '"')

+scenarios_to_check_in_alternate_regime=['de38cc6bbd455ab4_conflict_injection_and_occlusion_injection_0_-5.0_19993', '37590d51254e58dc_conflict_injection_and_occlusion_injection_3_0.0_53064', 'de38cc6bbd455ab4_conflict_injection_and_occlusion_injection_3_0.0_19993', '4f08b1e72b505e66_conflict_injection_and_occlusion_injection_0_natural_occlusion_5.0_64483', '16978efcc2615392_conflict_injection_and_occlusion_injection_0_5.0_49454', '81079a3f37ad5c9b_conflict_injection_and_occlusion_injection_0_0.0_48667', '4d354b05fe6c5668_conflict_injection_and_occlusion_injection_2_5.0_52804', '4d354b05fe6c5668_conflict_injection_and_occlusion_injection_3_5.0_52804', '7c5b326ec9ec5bfe_conflict_injection_and_occlusion_injection_3_5.0_20368', 'e80001adcba85f26_conflict_injection_and_occlusion_injection_6_5.0_20187', '90d9dad035075ecf_conflict_injection_and_occlusion_injection_1_5.0_19960', 'de38cc6bbd455ab4_conflict_injection_and_occlusion_injection_1_-5.0_19994', 'de38cc6bbd455ab4_conflict_injection_an

(raylet) [2024-03-06 02:25:48,045 E 458942 458954] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-03-06_00-31-10_957136_458634 is over 95% full, available space: 615840059392; capacity: 16125711347712. Object creation will fail if spilling is required.


output_folder_unocc = "../../data/nuplan/exp/exp/simulation/closed_loop_multiagent/2024.03.06.02.25.50"


## Run Second Regime

In [12]:
main_simulation(cfg_unocc)

Global seed set to 0


2024-03-06 02:25:50,037 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/main_callback_builder.py:20}  Building MultiMainCallback...
2024-03-06 02:25:50,058 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/main_callback_builder.py:35}  Building MultiMainCallback: 4...DONE!
2024-03-06 02:25:50,221 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/worker_pool_builder.py:19}  Building WorkerPool...
2024-03-06 02:25:50,222 WARNING {/home/ehdykhne/nuplan-devkit/nuplan/planning/utils/multithreading/worker_ray.py:135}  Ray is running, we will shut it down before starting again!
2024-03-06 02:25:53,820 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/utils/multithreading/worker_ray.py:78}  Starting ray local!


2024-03-06 02:25:55,539	INFO worker.py:1636 -- Started a local Ray instance.


2024-03-06 02:25:56,265 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/utils/multithreading/worker_pool.py:101}  Worker: RayDistributed
2024-03-06 02:25:56,266 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/utils/multithreading/worker_pool.py:102}  Number of nodes: 1
Number of CPUs per node: 32
Number of GPUs per node: 4
Number of threads across all nodes: 32
2024-03-06 02:25:56,266 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/worker_pool_builder.py:27}  Building WorkerPool...DONE!
2024-03-06 02:25:56,266 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/folder_builder.py:32}  Building experiment folders...
2024-03-06 02:25:56,266 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/folder_builder.py:35}  

	Folder where all results are stored: ../../data/nuplan/exp/exp/simulation/closed_loop_multiagent/2024.03.06.02.25.50

2024-03-06 02:25:56,268 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/folder_build

Ray objects: 100%|██████████| 32/32 [00:07<00:00,  4.01it/s]
(raylet) [2024-03-06 02:26:05,494 E 467588 467600] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-03-06_02-25-53_846650_458634 is over 95% full, available space: 615839453184; capacity: 16125711347712. Object creation will fail if spilling is required.


2024-03-06 02:26:14,374 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/simulation_builder.py:83}  Building metric engines...
2024-03-06 02:26:14,501 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/simulation_builder.py:85}  Building metric engines...DONE
2024-03-06 02:26:14,501 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/simulation_builder.py:89}  Building simulations from 499 scenarios...


(raylet) [2024-03-06 02:26:15,498 E 467588 467600] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-03-06_02-25-53_846650_458634 is over 95% full, available space: 615839436800; capacity: 16125711347712. Object creation will fail if spilling is required.


2024-03-06 02:26:20,361 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/simulation_builder.py:151}  Modyfing Scenarios...
[
	'0539ebb5245c50e4',
	'0ddd9da92c395ced',
	'114d78c106945fa4',
	'14322a75bc5b539d',
	'16978efcc2615392',
	'3380e8be77f956ca',
	'37590d51254e58dc',
	'4223c5519579545a',
	'4bb3d62efd555aec',
	'4d354b05fe6c5668',
	'4f08b1e72b505e66',
	'5d0a2d85408059a0',
	'70c382ad73d25fab',
	'7c5b326ec9ec5bfe',
	'7e606542d9565b11',
	'81079a3f37ad5c9b',
	'8a1f98f89e825c6e',
	'8f7146c52287508b',
	'908da0165fd1586b',
	'90d9dad035075ecf',
	'a02ec8ca7e0d59a7',
	'de38cc6bbd455ab4',
	'e80001adcba85f26',
	'ec1c624d44275107',
	'edfa1e16b8015936',
	'f6f98fb344275686'
]
2024-03-06 02:26:20,820 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/simulation_builder.py:203}  Created 89 modified scenarios from 499 scenarios, 165 of which were modifiable.
2024-03-06 02:26:20,820 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/simulation_builder.p

Ray objects:   0%|          | 0/89 [00:00<?, ?it/s](raylet) [2024-03-06 02:26:25,504 E 467588 467600] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-03-06_02-25-53_846650_458634 is over 95% full, available space: 615839318016; capacity: 16125711347712. Object creation will fail if spilling is required.
(raylet) [2024-03-06 02:26:35,511 E 467588 467600] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-03-06_02-25-53_846650_458634 is over 95% full, available space: 615839244288; capacity: 16125711347712. Object creation will fail if spilling is required.
(raylet) [2024-03-06 02:26:45,518 E 467588 467600] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-03-06_02-25-53_846650_458634 is over 95% full, available space: 615839244288; capacity: 16125711347712. Object creation will fail if spilling is required.
(raylet) [2024-03-06 02:26:55,525 E 467588 467600] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-03-06_02-25-53_846650_458634 is over 95% ful

2024-03-06 02:32:51,435 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/simulation/runner/executor.py:129}  Number of successful simulations: 89
2024-03-06 02:32:51,436 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/simulation/runner/executor.py:130}  Number of failed simulations: 0
2024-03-06 02:32:51,436 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/utils.py:147}  Finished executing runners!
2024-03-06 02:32:51,445 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/utils.py:74}  Saved runner reports to ../../data/nuplan/exp/exp/simulation/closed_loop_multiagent/2024.03.06.02.25.50/runner_report.parquet
2024-03-06 02:32:51,446 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/simulation/main_callback/time_callback.py:27}  Simulation duration: 00:07:01 [HH:MM:SS]
2024-03-06 02:32:52,514 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/simulation/main_callback/metric_file_callback.py:79}  Metric files integration: 00:00:01 [HH:MM:SS]
2024-03-06 02:32:52,588 

## Prepare the nuBoard config

In [13]:
# output_folder_alt = []
# output_folder_alt.append(
#     "../../data/nuplan/exp/exp/simulation/closed_loop_reactive_agents/2023.12.09.21.59.48"
# )
# output_folder_alt.append(
#     "../../data/nuplan/exp/exp/simulation/closed_loop_reactive_agents/2023.12.09.23.21.13"
# )
# output_folder_alt.append(
#     "../../data/nuplan/exp/exp/simulation/closed_loop_nonreactive_agents/2023.12.10.07.32.41"
# )
# output_folder_alt.append(
#     "../../data/nuplan/exp/exp/simulation/closed_loop_nonreactive_agents/2023.12.10.08.29.23"
# )
# output_folder_alt.append(
#     "../../data/nuplan/exp/exp/simulation/closed_loop_reactive_agents/2023.12.11.00.41.30"
# )
# output_folder_alt.append(
#     "../../data/nuplan/exp/exp/simulation/closed_loop_reactive_agents/2023.12.10.09.31.44"
# )
# output_folder_alt.append(
#     "../../data/nuplan/exp/exp/simulation/closed_loop_reactive_agents/2023.12.13.12.21.28"
# )
# output_folder_alt.append(
#     "../../data/nuplan/exp/exp/simulation/closed_loop_reactive_agents/2023.12.13.13.07.48"
# )

In [14]:
# scenario_builder = "val"  # [nuplan (uses trainval), nuplan_mini, test, val, train_boston, train_pittsburgh, train_singapore]
# output_folder_occ = "../../data/nuplan/exp/exp/simulation/closed_loop_multiagent/2024.02.28.20.06.45"
# output_folder_unocc = "../../data/nuplan/exp/exp/simulation/closed_loop_multiagent/2024.02.28.20.55.22"
# output_folder_occ = (
#     "../../data/nuplan/exp/exp/simulation/closed_loop_multiagent/2024.03.05.19.32.17"
# )
# output_folder_unocc = (
#     "../../data/nuplan/exp/exp/simulation/closed_loop_multiagent/2024.03.05.20.12.11"
# )

output_folder = [output_folder_occ, output_folder_unocc]
print(output_folder_occ)
print(output_folder_unocc)

CONFIG_PATH = "../nuplan/planning/script/config/nuboard"
CONFIG_NAME = "default_nuboard"

# Initialize configuration management system
hydra.core.global_hydra.GlobalHydra.instance().clear()  # reinitialize hydra if already initialized
hydra.initialize(config_path=CONFIG_PATH)

# Compose the configuration
cfg = hydra.compose(
    config_name=CONFIG_NAME,
    overrides=[
        f"scenario_builder={scenario_builder}",  # set the database (same as simulation) used to fetch data for visualization
        f"simulation_path={output_folder}",  # [output_folder, output_folder_alt] nuboard file path(s), if left empty the user can open the file inside nuBoard
    ],
)

../../data/nuplan/exp/exp/simulation/closed_loop_multiagent/2024.03.06.00.31.10
../../data/nuplan/exp/exp/simulation/closed_loop_multiagent/2024.03.06.02.25.50


## Launch nuBoard (open in new tab - recommended)


In [15]:
from nuplan.planning.script.run_nuboard import main as main_nuboard

# Run nuBoard
main_nuboard(cfg)

2024-03-06 02:32:53,006 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/scenario_building_builder.py:18}  Building AbstractScenarioBuilder...
2024-03-06 02:32:53,023 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/scenario_building_builder.py:21}  Building AbstractScenarioBuilder...DONE!
2024-03-06 02:32:53,024 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/nuboard/nuboard.py:84}  Opening Bokeh application on http://localhost:5006/
2024-03-06 02:32:53,024 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/nuboard/nuboard.py:85}  Async rendering is set to: True
2024-03-06 02:32:53,024 INFO {/home/ehdykhne/miniconda3/envs/nuplan/lib/python3.9/site-packages/bokeh/server/server.py:403}  Starting Bokeh server version 2.4.3 (running on Tornado 6.3.3)
2024-03-06 02:32:53,025 WARNING {/home/ehdykhne/miniconda3/envs/nuplan/lib/python3.9/site-packages/bokeh/server/util.py:145}  Host wildcard '*' will allow connections originating from multiple (or possib

2024-03-06 02:32:53,954 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/nuboard/base/simulation_tile.py:172}  Minimum frame time=0.017 s
2024-03-06 02:32:54,226 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/nuboard/tabs/scenario_tab.py:485}  Rending scenario plot takes 0.0067 seconds.
2024-03-06 02:32:54,327 INFO {/home/ehdykhne/miniconda3/envs/nuplan/lib/python3.9/site-packages/tornado/web.py:2344}  200 GET / (127.0.0.1) 733.68ms


INFO:tornado.access:200 GET / (127.0.0.1) 733.68ms
INFO:tornado.access:101 GET /ws (127.0.0.1) 0.51ms


2024-03-06 02:32:54,926 INFO {/home/ehdykhne/miniconda3/envs/nuplan/lib/python3.9/site-packages/tornado/web.py:2344}  101 GET /ws (127.0.0.1) 0.51ms
2024-03-06 02:32:54,927 INFO {/home/ehdykhne/miniconda3/envs/nuplan/lib/python3.9/site-packages/bokeh/server/views/ws.py:132}  WebSocket connection opened
2024-03-06 02:32:54,927 INFO {/home/ehdykhne/miniconda3/envs/nuplan/lib/python3.9/site-packages/bokeh/server/views/ws.py:213}  ServerConnection created
2024-03-06 02:32:55,590 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/nuboard/base/simulation_tile.py:172}  Minimum frame time=0.017 s


(raylet) [2024-03-06 02:32:55,776 E 467588 467600] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-03-06_02-25-53_846650_458634 is over 95% full, available space: 614847938560; capacity: 16125711347712. Object creation will fail if spilling is required.
INFO:tornado.access:200 GET / (127.0.0.1) 714.96ms


...

(raylet) [2024-03-06 12:55:46,120 E 467588 467600] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-03-06_02-25-53_846650_458634 is over 95% full, available space: 614733398016; capacity: 16125711347712. Object creation will fail if spilling is required.
(raylet) [2024-03-06 12:55:56,123 E 467588 467600] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-03-06_02-25-53_846650_458634 is over 95% full, available space: 614733369344; capacity: 16125711347712. Object creation will fail if spilling is required.
(raylet) [2024-03-06 12:56:06,125 E 467588 467600] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-03-06_02-25-53_846650_458634 is over 95% full, available space: 614733287424; capacity: 16125711347712. Object creation will fail if spilling is required.
(raylet) [2024-03-06 12:56:16,128 E 467588 467600] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-03-06_02-25-53_846650_458634 is over 95% full, available space: 614733283328; capacity: 1612571

2024-03-06 12:57:02,794 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/nuboard/base/simulation_tile.py:575}  Frame deferred: 11
2024-03-06 12:57:02,805 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/nuboard/base/simulation_tile.py:902}  Processing render queue for frame 11
2024-03-06 12:57:02,988 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/nuboard/base/simulation_tile.py:575}  Frame deferred: 14
2024-03-06 12:57:03,806 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/nuboard/base/simulation_tile.py:902}  Processing render queue for frame 14


(raylet) [2024-03-06 12:57:06,141 E 467588 467600] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-03-06_02-25-53_846650_458634 is over 95% full, available space: 614733168640; capacity: 16125711347712. Object creation will fail if spilling is required.
(raylet) [2024-03-06 12:57:16,144 E 467588 467600] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-03-06_02-25-53_846650_458634 is over 95% full, available space: 614733168640; capacity: 16125711347712. Object creation will fail if spilling is required.
(raylet) [2024-03-06 12:57:26,146 E 467588 467600] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-03-06_02-25-53_846650_458634 is over 95% full, available space: 614733160448; capacity: 16125711347712. Object creation will fail if spilling is required.
(raylet) [2024-03-06 12:57:36,149 E 467588 467600] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-03-06_02-25-53_846650_458634 is over 95% full, available space: 614733160448; capacity: 1612571

2024-03-06 12:59:25,854 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/nuboard/tabs/scenario_tab.py:485}  Rending scenario plot takes 5.7303 seconds.


(raylet) [2024-03-06 12:59:26,181 E 467588 467600] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-03-06_02-25-53_846650_458634 is over 95% full, available space: 614732931072; capacity: 16125711347712. Object creation will fail if spilling is required.
(raylet) [2024-03-06 12:59:36,184 E 467588 467600] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-03-06_02-25-53_846650_458634 is over 95% full, available space: 614732922880; capacity: 16125711347712. Object creation will fail if spilling is required.
(raylet) [2024-03-06 12:59:46,188 E 467588 467600] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-03-06_02-25-53_846650_458634 is over 95% full, available space: 614732922880; capacity: 16125711347712. Object creation will fail if spilling is required.
(raylet) [2024-03-06 12:59:56,191 E 467588 467600] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-03-06_02-25-53_846650_458634 is over 95% full, available space: 614732898304; capacity: 1612571